In [1]:
import os
import glob
from PIL import Image
# from ultralytics import YOLO


In [2]:
# Total
total_train_images_folder = "/kaggle/input/totaltextstr/Total-Text/Train"
total_train_labels_folder = "/kaggle/input/totaltextstr/Total-Text/Annotation/groundtruth_polygonal_annotation/Train"

total_test_images_folder = "/kaggle/input/totaltextstr/Total-Text/Test"
total_test_labels_folder = "/kaggle/input/totaltextstr/Total-Text/Annotation/groundtruth_polygonal_annotation/Test"

total_train_output_images_folder = "/kaggle/working/train/images"
total_train_output_labels_folder = "/kaggle/working/train/labels"

total_test_output_images_folder = "/kaggle/working/test_total/images"
total_test_output_labels_folder = "/kaggle/working/test_total/labels"

In [3]:
def Total_process_annotations(train_images_folder, train_labels_folder, train_output_images_folder, train_output_labels_folder):
    # Tạo các thư mục đầu ra nếu chưa tồn tại
    os.makedirs(train_output_images_folder, exist_ok=True)
    os.makedirs(train_output_labels_folder, exist_ok=True)

    # Hàm chuyển đổi tọa độ polygon sang YOLO bounding box
    def polygon_to_yolo_bbox(x, y, width_img, height_img):
        x_min, x_max = min(x), max(x)
        y_min, y_max = min(y), max(y)
        x_center = (x_min + x_max) / 2 / width_img
        y_center = (y_min + y_max) / 2 / height_img
        bbox_width = (x_max - x_min) / width_img
        bbox_height = (y_max - y_min) / height_img
        return x_center, y_center, bbox_width, bbox_height

    # Duyệt qua các file annotation
    annotation_files = glob.glob(os.path.join(train_labels_folder, "*.txt"))

    for anno_file in annotation_files:
        base_name = os.path.basename(anno_file).replace("poly_gt_", "").replace(".txt", "")
        image_path = os.path.join(train_images_folder, f"{base_name}.jpg")
        
        try:
            # Mở ảnh để lấy kích thước
            img = Image.open(image_path)
            width_img, height_img = img.size

            # Lưu ảnh vào thư mục đầu ra
            output_image_path = os.path.join(train_output_images_folder, f"{base_name}.jpg")
            
            # Biến kiểm tra xem có bounding box hợp lệ không
            valid_boxes = False
            
            # Lưu file annotation mới vào thư mục đầu ra
            output_label_file = os.path.join(train_output_labels_folder, f"{base_name}.txt")
            
            with open(anno_file, "r") as f, open(output_label_file, "w") as out_f:
                for line in f:
                    try:
                        # Tách các phần tử trong dòng annotation và bỏ qua các phần không cần thiết
                        line = line.strip()
                        if "x: [[" not in line or "y: [[" not in line:
                            continue

                        # Lấy phần tọa độ x và y
                        x_part = line[line.index("x: [[") + 5: line.index("]], y:")]
                        y_part = line[line.index("y: [[") + 5: line.index("]], ornt:")]

                        # Chuyển đổi tọa độ x và y thành danh sách số
                        x = list(map(int, x_part.split()))
                        y = list(map(int, y_part.split()))

                        # Bỏ qua nếu số điểm không đủ
                        if len(x) < 4 or len(y) < 4:
                            continue
                        
                        # Tính toán bounding box YOLO
                        x_center, y_center, bbox_width, bbox_height = polygon_to_yolo_bbox(x, y, width_img, height_img)
                        
                        # Nếu bounding box hợp lệ, đánh dấu
                        if bbox_width > 0 and bbox_height > 0:
                            valid_boxes = True
                        
                        # Gán class_id (ví dụ: 0 cho tất cả các lớp trong OCR)
                        class_id = 0
                        
                        # Ghi vào file annotation
                        out_f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
                    except Exception as e:
                        print(f"Lỗi xử lý dòng trong file {anno_file}: {line}\nChi tiết lỗi: {e}")
            
            # Chỉ lưu ảnh và file annotation nếu có ít nhất một bounding box hợp lệ
            if valid_boxes:
                img.save(output_image_path)  # Lưu ảnh vào thư mục output
            else:
                os.remove(output_label_file)  # Xóa file label nếu không có bounding box hợp lệ
                print(f"Không có bounding box hợp lệ cho ảnh {base_name}.jpg, file annotation không được lưu.")
                
        except Exception as e:
            print(f"Lỗi xử lý file: {anno_file}\nChi tiết lỗi: {e}")

In [4]:
Total_process_annotations(total_train_images_folder, total_train_labels_folder, total_train_output_images_folder, total_train_output_labels_folder)
Total_process_annotations(total_test_images_folder, total_test_labels_folder, total_test_output_images_folder, total_test_output_labels_folder)


Lỗi xử lý file: /kaggle/input/totaltextstr/Total-Text/Annotation/groundtruth_polygonal_annotation/Train/poly_gt_img61.txt
Chi tiết lỗi: [Errno 2] No such file or directory: '/kaggle/input/totaltextstr/Total-Text/Train/img61.jpg'
